In [18]:
import numpy as np
import pandas as pd
from django.db.models import Sum
from public_data.models import OcsgeDiff, CouvertureSol
from project.models import Project

In [2]:
project = Project.objects.get(pk=6)
project

<Project: Diagnostic de CA Grand Auch Coeur de Gascogne>

In [14]:
cities = CommuneDiff.objects.filter(city__in=project.cities.all())
cities

<QuerySet [<CommuneDiff: CommuneDiff object (16)>, <CommuneDiff: CommuneDiff object (88)>, <CommuneDiff: CommuneDiff object (90)>, <CommuneDiff: CommuneDiff object (82)>, <CommuneDiff: CommuneDiff object (56)>, <CommuneDiff: CommuneDiff object (116)>, <CommuneDiff: CommuneDiff object (15)>, <CommuneDiff: CommuneDiff object (21)>, <CommuneDiff: CommuneDiff object (59)>, <CommuneDiff: CommuneDiff object (26)>, <CommuneDiff: CommuneDiff object (111)>, <CommuneDiff: CommuneDiff object (75)>, <CommuneDiff: CommuneDiff object (7)>]>

In [13]:
headers = CouvertureSol.objects.all().values_list("code_prefix", flat=True)
headers

<QuerySet ['CS1', 'CS1.1', 'CS1.1.1', 'CS1.1.1.1', 'CS1.1.1.2', 'CS1.1.2', 'CS1.1.2.1', 'CS1.1.2.2', 'CS1.2', 'CS1.2.1', 'CS1.2.2', 'CS1.2.3', 'CS2', 'CS2.1', 'CS2.1.1', 'CS2.1.1.1', 'CS2.1.1.2', 'CS2.1.1.3', 'CS2.1.2', 'CS2.1.3', '...(remaining elements truncated)...']>

In [23]:
qs = (
    OcsgeDiff.objects.intersect(project.combined_emprise)
    .filter(year_old__gte=project.analyse_start_date, year_new__lte=project.analyse_end_date)
    .values("cs_new", "cs_old")
    .annotate(total=Sum("surface") / 10000)
    .order_by("cs_new", "cs_old")
)
qs

<QuerySet [{'cs_new': 'CS1.1.1.1', 'cs_old': 'CS1.1.1.1', 'total': Decimal('1.35867593000000000000')}, {'cs_new': 'CS1.1.1.1', 'cs_old': 'CS1.1.1.2', 'total': Decimal('0.81736174000000000000')}, {'cs_new': 'CS1.1.1.1', 'cs_old': 'CS1.1.2.1', 'total': Decimal('1.23240790000000000000')}, {'cs_new': 'CS1.1.1.1', 'cs_old': 'CS2.1.1.1', 'total': Decimal('0.38558054000000000000')}, {'cs_new': 'CS1.1.1.1', 'cs_old': 'CS2.1.1.3', 'total': Decimal('0.00245791000000000000')}, {'cs_new': 'CS1.1.1.1', 'cs_old': 'CS2.1.2', 'total': Decimal('0.08931012000000000000')}, {'cs_new': 'CS1.1.1.1', 'cs_old': 'CS2.2.1', 'total': Decimal('15.3646991000000000')}, {'cs_new': 'CS1.1.1.2', 'cs_old': 'CS1.1.1.1', 'total': Decimal('0.12621310000000000000')}, {'cs_new': 'CS1.1.1.2', 'cs_old': 'CS1.1.1.2', 'total': Decimal('0.62289064000000000000')}, {'cs_new': 'CS1.1.1.2', 'cs_old': 'CS1.1.2.1', 'total': Decimal('0.88423287000000000000')}, {'cs_new': 'CS1.1.1.2', 'cs_old': 'CS2.1.1.1', 'total': Decimal('0.259786590

In [27]:
pd.options.display.float_format = '${:,.2f}'.format
df = (
    pd.DataFrame(diff)
    .pivot(index='cs_old', columns='cs_new', values='total')
    .fillna(0)
    
)
df

cs_new,CS1.1.1.1,CS1.1.1.2,CS1.1.2.1,CS1.2.2,CS2.1.1.1,CS2.1.1.3,CS2.1.2,CS2.2.1
cs_old,,,,,,,,
CS1.1.1.1,1.35867593000000000000,0.12621310000000000000,0.35150908000000000000,0,0.66334341000000000000,0,0,1.66179848000000000000
CS1.1.1.2,0.81736174000000000000,0.62289064000000000000,0.41641198000000000000,0,0.000077450000000000000000,0,0,0.37489375000000000000
CS1.1.2.1,1.23240790000000000000,0.88423287000000000000,1.88614463000000000000,0,0.11218000000000000000,0.000018980000000000000000,0,8.3765896300000000
CS1.2.2,0,0,0,0,0,0,0,0.41868588000000000000
CS2.1.1.1,0.38558054000000000000,0.25978659000000000000,2.9036345400000000,0,2.2647647400000000,0,0,13.8537247400000000
CS2.1.1.2,0,0,0.97386572000000000000,0,0,0,8.8286243500000000,0
CS2.1.1.3,0.00245791000000000000,0,0.00119921000000000000,0,0,0,0,0
CS2.1.2,0.08931012000000000000,0.01679606000000000000,5.1228272700000000,0,0,0,0,2.5966695400000000
CS2.1.3,0,0,0,0,0,0,0,5.9588693100000000


In [46]:
print(" "*10, end="|")
for col in df.columns:
    print(f"{col: >10}", end=" |")
print()
for index, row in df.iterrows():
    print(f"{index: >10}", end="|")
    for col in df.columns:
        print(f"{row[col]: >10.2f}", end=" |")
    print()

          | CS1.1.1.1 | CS1.1.1.2 | CS1.1.2.1 |   CS1.2.2 | CS2.1.1.1 | CS2.1.1.3 |   CS2.1.2 |   CS2.2.1 |
 CS1.1.1.1|      1.36 |      0.13 |      0.35 |      0.00 |      0.66 |      0.00 |      0.00 |      1.66 |
 CS1.1.1.2|      0.82 |      0.62 |      0.42 |      0.00 |      0.00 |      0.00 |      0.00 |      0.37 |
 CS1.1.2.1|      1.23 |      0.88 |      1.89 |      0.00 |      0.11 |      0.00 |      0.00 |      8.38 |
   CS1.2.2|      0.00 |      0.00 |      0.00 |      0.00 |      0.00 |      0.00 |      0.00 |      0.42 |
 CS2.1.1.1|      0.39 |      0.26 |      2.90 |      0.00 |      2.26 |      0.00 |      0.00 |     13.85 |
 CS2.1.1.2|      0.00 |      0.00 |      0.97 |      0.00 |      0.00 |      0.00 |      8.83 |      0.00 |
 CS2.1.1.3|      0.00 |      0.00 |      0.00 |      0.00 |      0.00 |      0.00 |      0.00 |      0.00 |
   CS2.1.2|      0.09 |      0.02 |      5.12 |      0.00 |      0.00 |      0.00 |      0.00 |      2.60 |
   CS2.1.3|      0.00 |     

In [47]:
list(df.columns)

['CS1.1.1.1',
 'CS1.1.1.2',
 'CS1.1.2.1',
 'CS1.2.2',
 'CS2.1.1.1',
 'CS2.1.1.3',
 'CS2.1.2',
 'CS2.2.1']

In [50]:
result = {
    index: {col: row[col] for col in df.columns} for index, row in df.iterrows()
}
result

{'CS1.1.1.1': {'CS1.1.1.1': Decimal('1.35867593000000000000'),
  'CS1.1.1.2': Decimal('0.12621310000000000000'),
  'CS1.1.2.1': Decimal('0.35150908000000000000'),
  'CS1.2.2': 0,
  'CS2.1.1.1': Decimal('0.66334341000000000000'),
  'CS2.1.1.3': 0,
  'CS2.1.2': 0,
  'CS2.2.1': Decimal('1.66179848000000000000')},
 'CS1.1.1.2': {'CS1.1.1.1': Decimal('0.81736174000000000000'),
  'CS1.1.1.2': Decimal('0.62289064000000000000'),
  'CS1.1.2.1': Decimal('0.41641198000000000000'),
  'CS1.2.2': 0,
  'CS2.1.1.1': Decimal('0.000077450000000000000000'),
  'CS2.1.1.3': 0,
  'CS2.1.2': 0,
  'CS2.2.1': Decimal('0.37489375000000000000')},
 'CS1.1.2.1': {'CS1.1.1.1': Decimal('1.23240790000000000000'),
  'CS1.1.1.2': Decimal('0.88423287000000000000'),
  'CS1.1.2.1': Decimal('1.88614463000000000000'),
  'CS1.2.2': 0,
  'CS2.1.1.1': Decimal('0.11218000000000000000'),
  'CS2.1.1.3': Decimal('0.000018980000000000000000'),
  'CS2.1.2': 0,
  'CS2.2.1': Decimal('8.3765896300000000')},
 'CS1.2.2': {'CS1.1.1.1': 0,

In [65]:
headers = [_ for _ in list(result.values())[0].keys()]
headers

['CS1.1.1.1',
 'CS1.1.1.2',
 'CS1.1.2.1',
 'CS1.2.2',
 'CS2.1.1.1',
 'CS2.1.1.3',
 'CS2.1.2',
 'CS2.2.1']

dict_keys(['CS1.1.1.1', 'CS1.1.1.2', 'CS1.1.2.1', 'CS1.2.2', 'CS2.1.1.1', 'CS2.1.1.3', 'CS2.1.2', 'CS2.2.1'])